<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS207 Project Milestone 2  - @Software-Samurais</h1> 
    
### Group 3: 
#### Erick Ruiz, Jingyuan Liu, Kailas Amin, Simon (Xin) Dong



<hr style='height:2px'>

In [1]:
from IPython.display import Latex


# 1 Introduction

The increasing importance of computational models in science and business alongside the slowing pace of advances in computing hardware has increased the need for efficient and accurate evaluations of derivatives. Many important applications such as simulation, opti- mization, and neural networks rely on repeated differentiation of complex functions.

Before the advent of automatic differentiation (AD) the primary method for derivative evaluation was the method of finite differences (FD), where the function to be evaluated is effectively treated as black box oracle.1 As the FD method is effectively sampling, the granularity (i.e. step size) of the algorithm can introduce error effects if it is either too large or too small, but even at the perfect medium, f′(x) evaluations cannot reach machine precision. The alternative approach, fully symbolic differentiation (SD), is cumbersome and inefficient in many cases. In the case of a complex computer program, the size of the symbolic expression could grow to outrageous size and cause significant inefficiency.

The approach of algorithmic differentiation seeks to find the best of both worlds, with machine precision and easy evaluation. This is done by repeated evaluation of the chain rule at a point stored in a table called the computational trace. Thus rather than storing to full symbolic expression, an AD code only needs to apply the chain rule to a specific evalua- tion, representable by a single variable. This approach allows us to achieve the accuracy of symbolic approaches while drastically reducing the cost of evaluation.
Within the umbrella of automatic differentiation, we seek to implement the forward mode which evaluates the intermediate results directly in an inside out manner. Other approaches such as reverse mode also have specific advantages especially in the fields of machine learning and artificial intelligence– or in any context in which the number of inputs dominates the number of outputs.

The method of automatic differentiation, sometimes also referred to as algorithmic differ- entiation, addresses the weaknesses of the finite difference method by providing a systematic way to calculate derivatives numerically to arbitrary precision. The goal of AutoDiff is to implement the forward mode of automatic differentiation, as it is a relevant feature that even some mainstream machine learning libraries, such as PyTorch, lack.


# 2 Background
Understanding the concept of a derivative is crucial to all aspiring and
practicing scientists, engineers, and mathematicians. It is one of the
first concepts introduced in first-year calculus courses at all
universities. The idea is simple. Given a function, $f(x)$, how can we
quantify the rate of change of the function due to an infinitesimal
change, $\Delta x$, in the argument, $x$? The answer is typically given
in terms of the limit definition of the derivative.
\begin{equation}
f'(x) = \lim_{\Delta x\rightarrow 0} \frac{f(x+\Delta x)-f(x)}{\Delta x}
\tag{1}
\end{equation}

While equation (1) holds for any function, in practice, it is
easier to calculate derivatives analytically according to a set of
rules. However, obtaining an analytical expression for the derivative
becomes exceedingly difficult if the function of interest is composed of
many elementary functions.\
In any case of a derivative evaluating program, we will need to create a
code which takes as inputs both a function and evaluation point and then
return the derivative at that point. There are many approaches to
perform this calculation.\
For example, consider the following function.

\begin{equation}
f(x) = \exp\left[\frac{\sqrt{x^3 - \ln x + \sin(4x^2)}}{\cos(3x^5)}\right]
\tag{2}
\end{equation}

Calculating the first derivative would result
in the following expression. 
\begin{equation}
\begin{aligned}
    f'(x) &= \exp\left[\frac{\sqrt{x^3 - \ln x + \sin(4x^2)}}{\cos(3x^5)}\right]
    \sec^2(3x^5)\dots\nonumber\\
    &\qquad\times\left\{
    \frac{\cos(3x^5)}{2\sqrt{x^3 - \ln x + \sin(4x^2)}}
    \left[3x^2-\frac{1}{x}+8x\cos(4x^2)\right]\dots\right.\nonumber\\
    &\qquad\qquad+
    \left.\vphantom{\frac{\cos(3x^5)}{2\sqrt{x^3 - \ln x + \sin(4x^2)}}}
    15x^4\sin(3x^5)\sqrt{x^3 - \ln x + \sin(4x^2)}
    \right\}\end{aligned}
\tag{3}
\end{equation}

Although feasible, successive calculations
become more and more complex, and in practice, the quantity to be
differentiated may not be a function in closed-form but rather a set of
measurements or values given as a one-dimensional vector of numbers. In
that case, equation (1) can be approximated using the finite difference
method, which replaces an infinitesimal change in the argument for a
finite change. To show how this works, let us write the Taylor series
expansion of an arbitrary function, $f(x)$, at the point $x+h$.

\begin{equation}
f(x+h) = f(x) + hf'(x) + \frac{h^2}{2}f''(x) + \dots
    \tag{4}
\end{equation}

Keeping only terms of $\mathcal{O}(h)$ leaves us
with 

\begin{equation}
f(x) \approx f(h) + hf'(x),
\tag{5}
\end{equation}
which we can rearrange to write an
approximate expression for the derivative, $f'(x)$.

\begin{equation}
f'(x) \approx \frac{f(x+h)-f(x)}{h}
    \tag{6}
\end{equation}

The finite change, $h$, is called the step size, and
equation (6) is known as the forward difference. Its geometric interpretation is
described in Figure 1.<img src="./img/figure1.jpeg" width = "600" alt="name1" align=center />

Although the finite difference method is useful and easy to implement,
its accuracy can vary depending on the step size that is chosen. Suppose
we wish to approximate the derivative of $f(x)=\ln x$ using the forward
difference method described in equation (6) using step
sizes $h=\{10^{-1},\,
10^{-7},\,10^{-15}\}$. This is rather unnecessary because the analytical
derivative is just $f'(x) = 1/x$, but this example will serve to
illustrate the drawbacks of the finite difference method. At
$h=10^{-1}$, the numerical derivative is inaccurate because the step
size is too large, making the calculations susceptible to truncation
error. Conversely, at $h=10^{-15}$, the forward difference method also
gives inaccurate results because the calculations can only be
represented to a finite precision by the hardware in use. Hence,
rounding error also affects the stability of the finite difference
method.

In order to evaluate the derivative via forward mode AD we first
construct a computational graph which encodes the composition and
dependence of sub-function evaluations. It is important to note that
each elementary function evaluation needs to be understood at a symbolic
level! After constructing the graph, the chain rule is simply applied
successively to evaluations at a single point which then generates a
table known as the computational trace. To illustrate the concept,
consider the following example, adapted from *Evaluating derivatives:
principles and techniques of algorithmic differentiation* by Griewank
and Walther.
\begin{equation}
f(x,y) = \left[\sin\left(\frac{x}{y}\right) + \frac{x}{y} - \exp(y)\right]
    \left[\frac{x}{y} - \exp(y)\right]
    \tag{7}
\end{equation}
A function of two arguments (i.e. variables $x$ and
$y$) like the one in equation (7) can be
evaluated at point by replacing the arguments with numerical values. The
series of calculations needed to carry out the evaluation can be
visualized as a computation graph, as shown in Figure 2.<img src="./img/figure2.png" width = "800" alt="name" align=center />
The graph helps visualize the order of the
computations, and it also establishes the dependence of successive
calculations on previous ones.\
Although in many cases, the AD approach is given the entire function as
input, this is not always the case. This necessitates the need for a
\"seed\" value. In the basic case when the entire function is given, we
can simply initialize the computational trace (discussed below) as 1.
However, given the need for modularity which respects the chain rule, it
is also possible to input a different seed, allowing for the integration
of AD onto a sub segment of the function.

\
The procedure for constructing such a graph is the following. Define a
node for each of the inputs using a new variable. For this example, we
will let $x_1 = x$ and $x_2 = y$. From there, any successive nodes may
accept two inputs at maximum, and each node represents a new
calculation. For example, in the computational graph shown in Figure 3, the node $x_3$ represents the calculation
$x_1/x_2$, which later becomes an input in successive nodes. While the
computational graph is useful for visualizing the entire computation
procedure, the computational trace is useful for storing values as the
computation is carried out. The computational trace for
$f(1.5000, 0.5000)$ is given in Table 1![Table1](img/table1.png) In reference to the above, we can note that in
this trace, the seed is 1 and thus suppressed as is often notated. In
addition, it is important to note that depending on what implementation
of forward mode the exact target computed can be slightly different. In
particular, the vectorized extension of our milestone 2 code computes
the Jacobian.

Beyond the basic forward mode, there exists other implementations of AD
in the so called "reverse mode\". This approach can increase efficiency
significantly in cases when the number of inputs is far greater than the
number of outputs.


# 3 How to use

## 3.1 <span id="jump">Installation</span>

The `github` url to the project is https://github.com/Software-Samurais/cs207-FinalProject.
####  - Download the package from GitHub to your folder via these commands (in the terminal)
Assuming that the user already has the latest version of `Python` 3, `Git` and a package manager of choice installed (like `pip`).



```shell
# Clone the repo
git clone https://github.com/Software-Samurais/cs207-FinalProject.git

```

#### - Create a virtual environment and activate it (for Mac and Ubutun)

```shell
# If you don't have virtualenv, install it
sudo easy_install virtualenv
# Create virtual environment
virtualenv env
# Activate your virtual environment
source env/bin/activate
```

#### -  Install the necessary dependencies :

```shell
pip install -r requirements.txt
```

#### - Run module tests (in the root directory)
```shell
# Run module tests if you like
python -m pytest ./test/test_forward.py
```

## 3.2 Basic Demo

In this part, we will give a basic demo for a quick guide to the user. 

Once AutoDiff is installed, the user must import it to be able to use it. The user will have the option to either import the entire library or to choose only a subset of modules, classes, or methods to import.

In this demo, we import the package from autodiff directory and give an alias AD to the package.

In [2]:
# Import package from parent directory 
import sys 
sys.path.append("..") 
import autodiff.AD as AD

### Declare Variables

- Denote constants

In [3]:
# Constant (scalar or vector): User do not need to initialize derivative, it will set to 1.0
x1 = AD.AutoDiff(3.0)
# Each object has two attributes: val and der 
print(x1.val, x1.der)
x1

3.0 1.0


Function value: 3.0, Derivative value: 1.0

- Denote scalar variables

In [4]:
# User can pass value and derivative of a variable
x2 = AD.AutoDiff(2.5, 2.0)
x2

Function value: 2.5, Derivative value: 2.0

### Elementary Operations

We can perform basic elementary operations on `AutoDiff` objects, just as we would with numbers. For example, addition, subtraction, multiplication, division, and power operations are all supported. Simply define an instance of the `AutoDiff` class to get started. 

In [5]:
x = AD.AutoDiff(3.0)
y = AD.AutoDiff(2.0, 0.5)

In [6]:
# Negative
print(-x)
print(-y)

Function value: -3.0, Derivative value: -1.0
Function value: -2.0, Derivative value: -0.5


In [7]:
# Addition
print(x + 3.)
print(x + y)

Function value: 6.0, Derivative value: 1.0
Function value: 5.0, Derivative value: 1.5


In [8]:
# Subtraction
print(x - 1.)
print(x - y)

Function value: 2.0, Derivative value: 1.0
Function value: 1.0, Derivative value: 0.5


In [9]:
# Multiplication
print(3.* x)
print(x * y)

Function value: 9.0, Derivative value: 3.0
Function value: 6.0, Derivative value: 3.5


In [10]:
# Division
print(x / 3.)
print(1 / x)
print(x / y)

Function value: 1.0, Derivative value: 0.3333333333333333
Function value: 0.3333333333333333, Derivative value: -0.1111111111111111
Function value: 1.5, Derivative value: 0.125


In [11]:
# Power
print(x ** 2)
print(y ** 3)

Function value: 9.0, Derivative value: 6.0
Function value: 8.0, Derivative value: 6.0


### Trigonometric Functions, Exponentials, and Logarithms

Other basic elementary operations, such as trigonometric functions, exponentials, and natural logarithms are also supported. Example uses of each of these are shown in the following cells. 

In [12]:
# Sine
print(AD.sin(x))
print(AD.sin(x*y))

Function value: 0.1411200080598672, Derivative value: -0.9899924966004454
Function value: -0.27941549819892586, Derivative value: 3.360596003276281


In [13]:
# Cosine
print(AD.cos(x))
print(AD.cos(x-y))

Function value: -0.9899924966004454, Derivative value: -0.1411200080598672
Function value: 0.5403023058681398, Derivative value: -0.42073549240394825


In [14]:
# Tangent
print(AD.tan(x))
print(AD.tan(x/y))

Function value: -0.1425465430742778, Derivative value: 1.020319516942427
Function value: 14.10141994717172, Derivative value: 24.98125556581156


In [15]:
# Exponential
print(AD.exp(x))
print(AD.exp(y)**3)

Function value: 20.085536923187668, Derivative value: 20.085536923187668
Function value: 403.42879349273517, Derivative value: 605.1431902391028


In [16]:
# Logarithm
print(AD.log(x))

Function value: 1.0986122886681098, Derivative value: 0.3333333333333333


### Customized Functions

Customized functions can be defined using these elementary operations. For example, suppose we wish to work with the function
\begin{equation}
    f(x) = \exp\left(\sin x^2\right) - x^4 + \log(x). 
\end{equation}
We may define $f(x)$ using the standard Python conventions. However, instead of using the `exp` and `sin` methods from Numpy, we must use the `exp` and `sin` methods defined in the `AD` module because only the latter accept instances of the `AutoDiff` class as inputs. 

In [17]:
def f(x):
    return AD.exp(AD.sin(x**2)) - x**4 + AD.log(x)

f(x)

Function value: -78.39137437130643, Derivative value: -115.9215797663472

## 3.3 Application: Newton's Method

Given an equation, $f(x)$, Newton's method allows us to find solutions to $f(x)=0$ iteratively. Starting with an initial guess, $x_0$, we evaluate the function and its derivative at $x_0$. If $|f(x_0)| < \epsilon$, where $\epsilon$ is some tolerance value much less than one, then the iteration stops. Otherwise, the value of the next guess, $x_1$, is obtained as follows.
\begin{equation}
    x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}
\end{equation}
Following the same scheme, the $n$-th iteration can be written as 
\begin{equation}
    x_n = x_{n-1} - \frac{f(x_{n-1})}{f'(x_{n-1})},
\end{equation}
where $x_{n-1}$ is the guess at the previous iteration.

Since Newton's method makes use of the derivative of the function of interest, we may use automatic differentiation to implement Newton's method! Traditionally, we might consider using the finite difference method to approximate $f'(x)$ or define it explicitly in our routine. There are drawbacks to each of these approaches. The finite difference method is easy to implement, but its accuracy is strongly dependent on choosing the right step size. If $f(x)$ is a simple function, then explicitly defining $f'(x)$ is not a problem. However, this is not always possible. With the `AD` module, there is no need to approximate or explicitly define $f'(x)$. As we operate on our initial guess, $x_0$, the value of the derivative is automatically calculated as well, making it easy to implement Newton's method.

In [18]:
def newton(f, x0, tol=1e-16, max_iter=100):
    """Solves f(x) = 0 using Newton's method.
    
    Args:
    =========
    f (function): Function of interest
    x0 (float): Initial guess
    tol (float): Tolerance value
    max_iter (int): Maximum number of iterations
    
    Returns:
    =========
    xn.val (float): Solution to f(x) = 0 if it exists
                    None if xn.der is zero or if the maximum number of 
                    iterations is reached without satisfying the stopping  
                    criteria
    """
    
    # Initial guess
    xn = x0
    
    for n in range(max_iter):
        
        # Calculate f(xn) and f'(xn) using the AutoDiff class
        fn = f(xn)
        
        # Stop iterating if |f(xn)| is less than the tolerance value and return 
        # the solution, xn
        if abs(fn.val) < tol:
            print(f"Found a solution after {n} iterations.")
            return xn.val
        
        # Check if the derivative is zero
        if fn.der == 0:
            raise ValueError("Encountered zero derivative. No solution.")
            
        # Update guess
        xn = xn - fn.val/fn.der
        
    # Stop iterating if no solution is found within the allowed number of 
    # iterations
    print("Exceeded maximum number of iterations.")
    return None

In [19]:
# Function of interest
def f(x):
    return x**2-x-1

# Initial guess   
x0 = AD.AutoDiff(1.0)

print(f"Solution: {newton(f, x0)}")

Found a solution after 6 iterations.
Solution: 1.618033988749895


# 4 Software Organization

## 4.1 Directory Structure
The package's directory will be structured as follows:
``` py
Ccs207-FinalProject
	__init__.py                                         #Initialization
    /autodiff                                           #Back-end source code
	    /config                                         #Configuration for the project
	    AD.py
    /gui                                                #Front-end source code
        /dist                                           #Static css, js etc.
        /template                                       #Web html files
        /img                                            #Images used for font-end
    /utils                                              #Preprocessing scripts
	/test                                               #Test cases
		test_vector.py
		test_forward.py
        test_reverse.py
	/docs                                               #Documentation and records
		milestone1.pdf
        milestone2.ipynb
	/demo
    requirements.txt                                    #Packages on which the program depends
    README.md                                           #Introduction for the project
```
##  4.2 Basic Modules and functionality

So far, we have simple forward mode for auto-differentiation in `AD` module.

- `AD`: This module contains our custom library for autodifferentiation. 
    - It includes functionality for a AutoDiff class that contains values and derivatives. In the class, we override the operator like `__repr__`, `__neg__`, `__add__`, `__radd__`, `__sub__`, `__rsub__`, `__mul__`, `__rmul__`, `__truediv__`, `__rtruediv__`, `__pow__`.
    - In addition, we define class-specific functions e.g., `sine`, `cosine`, `tangent`, `exponentiation`, `logarithm`. Thus the user could use our defined math function easily (as we use numpy).

We plan to include reverse mode and other modules for application in the later work:

- `optimize`: This module is designed to perform optimization. Users can define a custom function to optimize. For example, the first n−1 columns denote the features of the data, and the final column represents the labels. The user could specify the function to optimize as “mse”. Then, the function will find a local minimum of the mean squared error objective function. Finally, the module allows for static and animated plots for visualization.

- `rootfinding`: This module is designed to find roots of a given function. It includes Newton’s method. It also allows the user to visualize static or animated results for visualization.

    
## 4.3 Test Suite
Coding is the fundamental part of software development. Equally significant is build and testing. 
- We would utilize Travis CI and CodeCov to make the development process more reliable and professional. 

     `Travis CI` is used as a distributed CI (Continuous Integration) tools to build and automate test the project.
     
     `CodeCov` is used for test results analysis (eg. measuring test code coverage) and visu- alization.

We have already set up these integrations, with badges included in the README.md, showing that build passed 100%, and coverage 100% .
- Users can run the test suite by running in the root folder:
```shell
python -m pytest ./test/test_forward.py
```
- All test files will be placed in the test folder.

    `test_forward`: It includes tests for scalar functions to ensure that the AD module properly calculates values of scalar functions and gradients with respect to scalar inputs.
    
    we plan to add more test files in future work:

    `test_rootfinding`: This is a test suite for rootfinding.
    
    `test_optimize`: This is a test suite for optimization.



## 4.4 Software Package and Distribution:
### Package distribution
We will package our software using `PyPI` (Python Package Index) for release. Write and run ’setup.py’ to package the software and upload it to the distribution server, thus people in community could easily download our package by ’pip install’.
### Version Control
We will take Version Control into consideration according to the standard in Python Enhancement Proposal (PEP) 386. With version control, we can tell the user what changes we made and set clear boundaries for where those changes occurred.
### Framework
- For web development, we would use `Flask`, a micro web framework, which is suitable for a small team to complete the implementation of a feature-rich small website and easily add customized functions.
- For GUI (Graphical User Interface), we may choose `Vue.js`, a JavaScript frame- work for building user interfaces and single-page applications. Because it offers many API (Application Program Interface) to integrate with existing projects and is easy to get started. It is better in code reuse compared to frameworks like `jQuery`.

## 4.5 How to install the package

At this point, our package isn't on `PyPI` (will distribute later). User can download and install our package manually as mentioned in [3.1 Installation](#jump).



# 5 Implementation

## 5.1 Forward Mode

### Core Data Structures

In current implementation, we define our own class `AutoDiff` as the main data structure. For future work, we would use 
- **List and Numpy array** - for vectorization. Track how many independent variables have been created so far and store copies of these independent variables.

- **Graph** - for reverse mode. Every leaf variables (aside from constants) is an instance of Node. A computational graph consists of multiple nodes. Each node has two main attributes, self.inputs and self.op which indicate inputs of the current node and the operation between inputs. The input list is a python list whose elements are inputs of the current node. For example, if `node_3 =  ad.Op_1(node_2, node_1)` , we have `node_3.inputs = [node_2, node_1]` and `node_3.op = ad.Op_1`. Computation between leaf variables will be reloaded and return a new node. By doing this, we can parse the input equation into a computational graph automatically.

### Core Classes
- `AutoDiff`: The core class for the scalar implementation of forward mode automatic differentiation is the `AutoDiff` class. Defining an instance of the class allows the user to store and update function and derivative values. When an instance of the `AutoDiff` class is defined, the user is able to easily store,and update function and derivative values. These function and derivative values are attributes of the `AutoDiff` class.

#### Important Attributes
**Note:** The following two attributes are considered private. The user should not access these directly.
- `_val`: Stores the function value
- `_der`: Stores the derivative value; defaults to `1.0` if the user does not pass in a second argument to `AutoDiff` when defining a new instance

There exist getter and setter methods to access current values and set new values for `_val` and `_der`.

#### Methods
We have implemented in this milestone the following methods for the `AutoDiff` class:

- `__init__`:initialize a AutoDiff class object, regardless of the user input, with values and derivatives stored as float.
- `__repr__`: overload the print format, prints self in the form of Function value: [val] Derivative value: [der]) 
- `__neg__`: overload negitive function
- `__add__`: overload add function to handle addition of AutoDiff class objects and addition of AutoDiff and non-AutoDiff objects
- `__radd__`: preserve addition commutative property.
- `__sub__`: overload sub function to handle addition of AutoDiff class objects and addition of AutoDiff and non-AutoDiff objects
- `__rsub__`: preserve substraction commutative property.
- `__mul__`: overload multiplication function to handle addition of AutoDiff class objects and addition of AutoDiff and non-AutoDiff objects
- `__rmul__`: preserve multiplication commutative property.
- `__truediv__`: overload division function to handle addition of AutoDiff class objects and addition of AutoDiff and non-AutoDiff objects
- `__rtruediv__`: preserve division commutative property. 
-` __pow__`: extend power functions to AutoDiff class objects, power times should be a integer.


#### Functions

- `sin(x)`
- `cos(x)`
- `tan(x)`
- `exp(x)`
- `log(x)`

The funtions above take as input a AutoDiff class object and return a new AutoDiff class object with its value and derivative updated according to the elementary function it corresponds to using the chain rule.

### External Dependencies
- Numpy: is used for mathematical calculations. 

- pytest: it provides us a systematic way to test 

- TravisCI and Codecov: they are our test suites.

### Elementary Functions
The following elementary functions are supported: sine, cosine, tangent, exponential, and logarithm. Recall from elementary calculus that the derivatives of these functions are the following.
\begin{align}
    \frac{d}{dx} \sin x &= \cos x\\
    \frac{d}{dx} \cos x &= -\sin x\\
    \frac{d}{dx} \tan x &= \sec^2 x = \frac{1}{\cos^2 x}, \quad x \neq \frac{\pi}{2}\\
    \frac{d}{dx} \exp x &= \exp x\\
    \frac{d}{dx} \ln x &= \frac{1}{x}, \quad x > 0
\end{align}
Note that $\tan x$ and $\ln x$ will raise a `ValueError` if the user passes in a function value that is outside the valid domain.

The methods that execute these elementary functions are defined in the `AD` module and work as follows. Each method takes in a single argument that is an instance of the `AutoDiff` class and returns a new instance of the `AutoDiff` class with the appropriate function and derivative values. To avoid rounding error, the `check_tol` method in the `AD` module compares the calculated values with their rounded counterparts. For example, suppose we wish to calculate $\tan x$ and its derivative at $x=\pi/4$. The `check_tol` method ensures that `AD.tan(AD.AutoDiff(np.pi/4))` returns `Function value: 1.0, Derivative value: 2.0` rather than `Function value: 1.0, Derivative value: 1.999999...`.

## 5.2 Planning work

### Not implemented - Vectorizaton
In this current release, our automatic differentiation package can only handle scalar to scalar functions. In the future, we will extend this work to include the more general category of scalar to scalar functions. This extension will primarily preserve the structure of the code but will result in rewriting the main functions to be able to deal with vector inputs.

We are going to support gradient computation for vectors and matrixes.   For simplicity, we only discuss how to compute the gradient of the matrix because the vector is only the special case of a matrix.  Operations between matrixes have two classes. The first one is the element-wise operation. For this kind of operation, we do not have to change the code for scalar and only need to replace scalar values with `np.array`. The second one is the matrix-wise operation like matrix multiplication. For this kind of operation, we use `np.array` to represent the value of variables again. However, we need to reimplement the operations because they have different formulations of the gradient. For example, if the operation is matrix multiplication between matrix A and B, the returned gradient from output to A and B are $dA = dY \times B^T$ and $dB = A^T \times dY$.
Reverse mode


### Plan on implementing - Reverse mode, Optimization etc.
More details could be found at [6 Future Features](#6).


# 6 <span id='6'>Future Features </span>

These are some proposals for future features in the AD package. Some of these will be implemented in the final version!

### Reverse Mode
#### Overall Framework
For the reverse mode, we need to build the computational graph and figure out the sequence of computation correctly to get the right gradient.
#### Ststic V.S. Dynamic Graph
Basically, there are two strategies of building the computational graph, static graph and dynamic graph. The former computes value and gradient after finishing graph and the latter do the computation dynamically. Although dynamic is easier to debug and more user-friendly, static graph is more clear to understand reverse mode and more suitable for this course. So we choose static graph based reverse mode as our main framework.
#### Attributes and Data Structure Used in Class Node
A computational graph consists of multiple nodes. Each node has two main attributes, self.inputs and self.op which indicate inputs of current node and the operation between inputs. It is worth mentioning that nodes are created by their own operation. The input list is a python list whose elements are inputs of the current node. For example, if `node_3 =  ad.Op_1(node_2, node_1)` , we have `node_3.inputs = [node_2, node_1]` and `node_3.op = ad.Op_1`. Several operations are implemented, such as add, add_by_const, mul, mul_by_const, tan, sin and so on. 
#### How to Support Higher-Order Gradient
To better support higher order gradient computation, gradients of leaf and intermediate variable are also represented by nodes. As a result, our computational graph not only has nodes representing values of variable during forward propagation, but also has nodes representing gradients of variable during back propagation. If users want to compute higher order gradient, they can easily add new nodes representing gradient of nodes of low order gradient. With this mechanism, we can compute any order gradient.
#### Traveral on Graph
How to find the path of creating nodes of gradients in the graph is also quite important to get correct results efficiently. Given a list of nodes, we use a post-order Depth First Search (DFS) to get a topological sort list of nodes ending in our target nodes. The reverse of this list is the right sequence to add nodes of gradients.

#### Efficiency of Our Implementation
As we mentioned above, we use nodes to represent both values and gradients of variables in the same graph. This implementation has two advantages. First, to compute a gradient node, we may have to access to some associated value nodes. With our implementation, we do not have to recompute the results of these value nodes. Second, we do post-order DFS on the computational graph and try to find a topology-sorted list of nodes which means that we only need to use part of the graph to compute the result of a certain node we care about instead of executing the whole graph.


### Optimization


#### What to realize
Gradient Descent is used to find the local minimum of a function f by taking locally optimum steps in the direction of steepest descent. A common application is in machine learning when a user desires to find optimal weights to minimize a loss function.


BFGS, short for “Broyden–Fletcher–Goldfarb–Shanno algorithm”, seeks a stationary point of a function, i.e. where the gradient is zero. In quasi-Newton methods, the Hessian matrix of second derivatives is not computed. Instead, the Hessian matrix is approximated using updates specified by gradient evaluations (or approximate gradient evaluations).

Here is a pseudocode of the implementation of BFGS.<img src="./img/op1.jpeg" width = "400" alt="name" align=center />


#### Method
`GradientDescent:` solve for a local minimum of a function f:ℝm⇒ℝ1. If f is a convex function, then the local
minimum is a global minimum.

Input:

- f: In machine learning applications, this should be the cost/objective function. If f is a function of multiple scalars (i.e. ℝm⇒ℝ1), the arguments to f should be passed in as a numpy array. In this case, define f as follows:


```python
def f(variables):
    x, y, z = variables
    return x ** 2 + y ** 2 + z ** 2 + np.sin(x)
```

- x: int, float, or AD objects (multivariate). It is the initial guess for a root of f. If f is a scalar to scalar function (i.e. ℝ1⇒ℝ1), and the initial guess for the root is 1, then a valid x is x = 1. If f is a function of multiple scalars, with initial guess for the root as (1, 2, 3), then a valid definition of x is a numpy array.

- iters: int, optional, default=1000. The maximum number of iterations to run the Newton root finding algorithm. The algorithm will run for min (t,iters) iterations, where t is the number of steps until tol is satisfied.

- tol: float, optional, default=1e-8. If the difference is smaller than tol, then the algorithm will terminate, even if the number of iterations has not reached iters.

Output:

- minimum: AD obeject ∈ℝm. The val attribute contains a numpy array of the minimum that the algorithm found in min(iters,t) iterations (iters,t defined above). The der attribute contains the Jacobian value at the specified root.
- var_path: a numpy array (ℝn×m), where n=min(iters,t) is the number of steps of the algorithm and m if the dimension of the minimum, where rows of the array are steps taken in consecutive order.
- g_path: a numpy array (ℝn×1), containing the consecutive steps of the output of f at each guess in var_path.

`plot_result:` A plot function that can visulize the gradient descent process

Input: 
- f: the function we want to optimize
- var_pathh: a numpy array (ℝn×m), where n=min(iters,t) is the number of steps of the algorithm and m if the dimension of the minimum, where rows of the array are steps taken in consecutive order.
- g_path: a numpy array (ℝn×1), containing the consecutive steps of the output of f at each guess in var_path.
- f_string: string, title of the plot 
- x_lims, y_lims: two tuples, to set range of X-axes and Y-axes

Output: None, just plot a figure.


#### How to use
For example, we want to find minimum of quartic function f(x)=x4. 

```python
import AutoDiff.optimize as opt
import numpy as np
import matplotlib.pyplot as plt

def f(x):
        return x ** 4 + 2 * (x ** 3) - 12 * (x ** 2) - 2 * x + 6

# Function string to include in plot
f_string = 'f(x) = x^4 + 2x^3 -12x^2 -2x + 6'

x0 = 4
solution, xy_path, f_path = opt.GradientDescent(f, x0, iters=1000, eta=0.002)
# A plot function that can visulize the process
opt.plot_results(f, xy_path, f_path, f_string, x_lims=(-6, 5), y_lims=(-100, 70))
```

#### Changes to current version

- Directory structure: do not change a lot, just add files in autodiff directory
- New modules: add optimize module
- Classes: may add optimize class, and some method mentioned above
- data structures: basically use numpy array, also list

#### Challenges
- It's may be difficult to plot a figure when the input is high dimension, we will try to figure it out refer to some existing packages
- S